In [ ]:
# -*- coding: utf-8 -*-
# -*- authors : Vincent Roduit, Filippo Quadri -*-
# -*- date : 2024-05-03 -*-
# -*- Last revision: 2024-05-03 -*-
# -*- python version : 3.9.18 -*-
# -*- Description: Notebook that summarize results-*-

In [ ]:
#Prepare Google Colab Environment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/EE-451-IAPR/project/src

# <center> EE - 451 Image Analysis and Pattern recognition </center>
## <center> Ecole Polytechnique Fédérale de Lausanne </center>
### <center>Coin Challenge </center>
---

In [1]:
#Import libraries
import torch
import importlib
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch import nn

import warnings
warnings.filterwarnings("ignore")

In [5]:
#Import files
import constants
importlib.reload(constants)

#Classes
from data_classes.ref_data import refCoin
from data_classes.train_data import trainCoin
from data_classes.test_data import testCoin

#Functions
from visualization import *
from pickle_func import *
from pre_processing.process_func import *
from pre_processing.data_augmentation import *
from post_processing.data_formating import *
from post_processing.dataloader import *

#Models
from models.cnn import Basic_CNN, Advanced_CNN, CnnRadius
%load_ext autoreload
%autoreload 2


# 1. Load different Datasets

In [3]:
ref_data = refCoin(save=False)

Loading data from pickle files


In [38]:
train_data = trainCoin(save=False, load_from_pickle=False)

Loading data from pickle files


In [6]:
test_data = testCoin(save=False)

Loading data from pickle files


# 2 Processing Data

In [39]:
#Process all the images :
# 1. clean the images and find the contours
# 2. create the masked images
# 3. create the coin images
# 4. if save = True, save the images and the class in a result folder
train_data.proceed_data()

Finding contours
Creating masked images
Creating coin images


In [ ]:
ref_data.proceed_data()

In [7]:
test_data.proceed_data()

Finding contours
Creating masked images
Creating coin images


## 3.1 Fetch the labels

In [8]:
conversion_table = get_classes_conv_table()

In [50]:
# Associate the labels to the coins
coin_labels = get_coin_labels()

In [51]:
# Extract the images and the labels + create a dataframe that summarize the data
train_images_raw,train_radius_infos, labels, df_train_images_labels = create_train_data_structure(train_data.coins, train_data.contours_tuple, coin_labels,conversion_table)

In [9]:
test_imgs, test_radius_infos, df_test_images = create_test_data_structure(test_data.coins, test_data.contours_tuple)

In [12]:
save_pickle(test_imgs, 'test_imgs.pkl')
save_pickle(test_radius_infos, 'test_radius_infos.pkl')
save_pickle(df_test_images, 'df_test_images.pkl')

In [ ]:
# Optional : save the coins in ../data/results/coins_classified
#           the coins are separated in folders according to their class
save_coins_classified(df_train_images_labels, train_images_raw)

## 3.2 Data augmentation and train/validation split

In [33]:
# Create the splits (train and validation)
train_images, train_radius, train_labels, val_images, val_radius, val_labels = create_splits(train_images_raw, train_radius_infos, labels)

In [ ]:
# Augment the training set with rotations
train_images_aug, train_radius_aug, train_labels_aug = augment_set_rotations(train_images, train_radius, train_labels)

# Augment the training set with Gaussian blur
# train_images_aug, train_radius_aug, train_labels_aug = augment_blur(train_images_aug, train_labels_aug)

In [ ]:
#Optional : save the datasets in ../data/results/pickle_files
save_pickle(result=train_images_aug, file_name='train_images_aug_resized.pkl')
save_pickle(result=train_radius_aug, file_name='train_radius_aug.pkl')
save_pickle(result=train_labels_aug, file_name='train_labels_aug.pkl')

save_pickle(result=val_images, file_name='val_images_resized.pkl')
save_pickle(result=val_radius, file_name='val_radius.pkl')
save_pickle(result=val_labels, file_name='val_labels.pkl')

In [ ]:
# Optional : Load precomputed data
train_images_aug_resized = load_pickle('train_images_aug_resized.pkl')
train_images_labels_aug = load_pickle('train_labels_aug.pkl')
train_radius_aug = load_pickle('train_radius_aug.pkl')

val_images_resized = load_pickle('val_images_resized.pkl')
val_radius = load_pickle('val_radius.pkl')
val_images_labels = load_pickle('val_labels.pkl')

In [13]:
test_imgs = load_pickle('test_imgs.pkl')
test_radius_infos = load_pickle('test_radius_infos.pkl')
df_test_images = load_pickle('df_test_images.pkl')

# 4 Train Neural Network


## 4.1 Create Dataloader

In [ ]:
train_dataloader, val_dataloader = create_dataloader(
    train_images=train_images_aug_resized,
    train_labels=train_images_labels_aug,
    train_radius=train_radius_aug, #Set to None if not needed
    val_images=val_images_resized, #Set to None if not needed
    val_labels=val_images_labels,
    val_radius=val_radius)

## 4.2 Custom Basic CNN

In [ ]:
image_dim = train_images_aug_resized.shape[1]
num_classes = len(conversion_table)

In [ ]:
# Define the model
cnn = Basic_CNN(img_size=image_dim, num_classes=num_classes)

# Define the optimizer
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
# Define the scheduler
scheduler = ReduceLROnPlateau(
    optimizer, mode="min", factor=0.1, patience=2, verbose=True
)
criterion = nn.BCEWithLogitsLoss()

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    train_dataloader,
    val_dataloader,
)

## 4.3 Custom Advanced CNN

In [ ]:
# Define the model
cnn = Advanced_CNN(img_size=image_dim, num_classes=num_classes)

# Define the optimizer
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
# Define the scheduler
scheduler = ReduceLROnPlateau(
    optimizer, mode="min", factor=0.1, patience=2, verbose=True
)
criterion = nn.BCEWithLogitsLoss()

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    train_dataloader,
    val_dataloader,
)

## 4.3 Custom CNN with Radius Informations

In [ ]:
# Define the model
cnn = CnnRadius(img_size=image_dim, num_classes=num_classes)

# Define the optimizer
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
# Define the scheduler
scheduler = ReduceLROnPlateau(
    optimizer, mode="min", factor=0.1, patience=2, verbose=True
)

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    train_dataloader,
    val_dataloader,
)

## 4.4 RESNET-50

In [ ]:
from transformers import AutoImageProcessor, ResNetForImageClassification
from torchvision import transforms
from datasets import load_metric

# Step 2: Modify the final layer
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

# Replace the classifier
num_classes = 15
model.classifier = nn.Linear(model.classifier.in_features, num_classes)

# Step 3: Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(10):  # Adjust the number of epochs as needed
    model.train()
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{10}, Loss: {loss.item()}")

# Evaluation
model.eval()
metric = load_metric("accuracy")
for images, labels in val_dataloader:
    images, labels = images.to(device), labels.to(device)
    with torch.no_grad():
        outputs = model(images).logits
    predictions = torch.argmax(outputs, dim=1)
    metric.add_batch(predictions=predictions, references=labels)

accuracy = metric.compute()
print(f"Test Accuracy: {accuracy['accuracy']:.4f}")